In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import pathlib
tf.__version__

'2.0.0'

![Image text](http://zh.d2l.ai/_images/residual-block.svg)

In [26]:
class Residual(layers.Layer):
    def __init__(self,num_chanels,use_1x1_conv=False,strides=1,**kwargs):
        super(Residual,self).__init__(**kwargs)
        self.conv1 = layers.Conv2D(filters=num_chanels,kernel_size=3,padding="same",strides=strides)
        self.conv2 = layers.Conv2D(filters=num_chanels,kernel_size=3,padding="same")
        
        if use_1x1_conv:
            self.conv3 = layers.Conv2D(filters=num_chanels,strides=strides,kernel_size=1)
        else:
            self.conv3 = None
            
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()
        
    
    def call(self,x):
        Y = keras.activations.relu(self.bn1(self.conv1(x)))
        Y = self.bn2(self.conv2(Y))
        
        if self.conv3:
            x = self.conv3(x)
        
        return keras.activations.relu(tf.add(Y,x))
        

In [27]:
res = Residual(5,use_1x1_conv=True)
x = tf.random.normal(shape=(1,5,5,3))
res(x)

<tf.Tensor: id=5189, shape=(1, 5, 5, 5), dtype=float32, numpy=
array([[[[0.        , 0.        , 0.        , 1.0525084 , 1.40469   ],
         [1.792652  , 0.        , 0.44587052, 0.8304973 , 0.5589856 ],
         [0.        , 0.        , 0.        , 0.        , 0.7277136 ],
         [0.52829194, 0.33394304, 0.        , 0.6304153 , 0.68458104],
         [0.20289277, 0.        , 0.        , 0.01815442, 0.80198115]],

        [[0.        , 2.0795255 , 0.25523627, 1.8635278 , 0.        ],
         [1.208963  , 0.87992   , 0.45269763, 0.28367552, 0.        ],
         [0.        , 0.        , 0.47086075, 0.9963012 , 0.        ],
         [0.        , 0.54346305, 0.        , 0.52445257, 0.        ],
         [0.69520164, 0.15453506, 0.19637758, 0.11191265, 0.        ]],

        [[0.        , 0.        , 0.        , 0.        , 0.75999755],
         [0.        , 0.5269203 , 0.        , 0.25629878, 0.        ],
         [0.        , 0.        , 0.        , 0.15020207, 0.830908  ],
         [

In [28]:
res = Residual(3)
x = tf.random.normal(shape=(1,5,5,3))
res(x)

<tf.Tensor: id=5352, shape=(1, 5, 5, 3), dtype=float32, numpy=
array([[[[1.9288943 , 1.0995517 , 0.77469164],
         [0.81393814, 0.        , 0.07809567],
         [0.        , 1.5463235 , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.31324953, 0.        , 0.        ]],

        [[0.        , 0.        , 1.0434363 ],
         [1.7660038 , 0.        , 0.        ],
         [0.3596599 , 1.2341691 , 0.5523492 ],
         [0.17545742, 0.0952903 , 0.        ],
         [0.        , 0.        , 1.1075728 ]],

        [[0.        , 0.2707551 , 0.833319  ],
         [0.71844065, 1.0182515 , 0.        ],
         [0.        , 0.6529656 , 0.        ],
         [0.        , 0.        , 0.27398854],
         [1.6161797 , 0.03567117, 1.1163002 ]],

        [[0.        , 0.13964036, 0.        ],
         [0.77965486, 0.        , 0.        ],
         [0.85333556, 1.1911275 , 1.053676  ],
         [0.19504511, 0.8152349 , 0.        ],
         [0.        , 0.64812434, 0.  

In [29]:
def res_block(num_chanels,num_residual,first_block=False):
    net = keras.Sequential()
    for i in range(num_residual):
        if i == 0 and not first_block:
            net.add(Residual(num_chanels,use_1x1_conv=True,strides=2))
        else:
            net.add(Residual(num_chanels))
    return net

In [35]:
net = keras.Sequential(
[
    layers.InputLayer(input_shape=(224,224,3)),
    layers.Conv2D(64,kernel_size=7,strides=2,padding="same"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D(pool_size=3,strides=2,padding="same"),
    res_block(64,2,first_block=True),
    res_block(128,2),
    res_block(256,2),
    res_block(512,2),
    layers.GlobalAveragePooling2D(),
    layers.Flatten(),
    layers.Dense(10,activation="softmax")
]
)

In [33]:
net.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_164 (Conv2D)          (None, 112, 112, 64)      9472      
_________________________________________________________________
batch_normalization_142 (Bat (None, 112, 112, 64)      256       
_________________________________________________________________
activation_10 (Activation)   (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
sequential_36 (Sequential)   (None, 56, 56, 64)        148736    
_________________________________________________________________
sequential_37 (Sequential)   (None, 28, 28, 128)       526976    
_________________________________________________________________
sequential_38 (Sequential)   (None, 14, 14, 256)     

In [40]:
class Residual2(layers.Layer):
    def __init__(self,num_chanels,use_1x1_conv=False,strides=1,**kwargs):
        super(Residual2,self).__init__(**kwargs)
        self.conv1 = layers.Conv2D(filters=num_chanels,kernel_size=3,padding="same",strides=strides)
        self.conv2 = layers.Conv2D(filters=num_chanels,kernel_size=3,padding="same")
        
        if use_1x1_conv:
            self.conv3 = layers.Conv2D(filters=num_chanels,strides=strides,kernel_size=1)
        else:
            self.conv3 = None
            
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()
        
    
    def call(self,x):
        Y = self.conv1(keras.activations.relu(self.bn1(x)))
        
        Y = keras.activations.relu(self.bn2(Y))
        
        if self.conv3:
            x = self.conv3(x)
        
        return self.conv2(tf.add(Y,x))

In [44]:
res = Residual2(3)
x = tf.random.normal(shape=(1,5,5,3))
res(x)

<tf.Tensor: id=12008, shape=(1, 5, 5, 3), dtype=float32, numpy=
array([[[[ 0.64910334, -0.18547405, -0.7451763 ],
         [-0.8170706 , -0.8199671 ,  0.24060956],
         [ 0.74791706,  0.98832005,  1.240883  ],
         [-1.348957  , -0.4730491 , -1.2346729 ],
         [ 0.20103338, -0.11028451,  0.6014904 ]],

        [[ 0.5080365 , -0.5417027 , -0.8722546 ],
         [-0.3201287 ,  0.45071432,  0.8221087 ],
         [ 1.0815982 ,  0.5164368 ,  0.9283405 ],
         [-0.92870337,  1.533142  ,  0.08751443],
         [ 0.14096233,  0.13931507, -0.66158456]],

        [[ 0.36633945, -0.2027487 , -0.14680251],
         [ 0.38898647,  0.83766615,  1.1265107 ],
         [-0.19134319,  1.6288823 , -0.30406162],
         [ 2.1021802 , -0.3307534 ,  0.66083515],
         [-1.5764809 , -0.6248381 , -1.4138529 ]],

        [[-0.82577896, -1.4803147 , -2.3744457 ],
         [ 2.0652113 ,  0.04899463,  1.1195637 ],
         [ 0.16036567,  0.68191683,  0.9488291 ],
         [-2.1948783 ,  0.0953

In [46]:
res = Residual2(3,use_1x1_conv=True,strides=2)
x = tf.random.normal(shape=(1,5,5,5))
res(x)

<tf.Tensor: id=12388, shape=(1, 3, 3, 3), dtype=float32, numpy=
array([[[[-1.9071822 ,  1.0142379 , -0.52718616],
         [-1.7347728 , -0.18682872,  1.6458956 ],
         [-3.0223117 , -0.00806379,  0.86540025]],

        [[-1.8879466 , -1.1229068 , -0.64781904],
         [-0.99851525, -0.79939336,  1.982673  ],
         [-0.6213891 ,  2.814167  , -0.26968914]],

        [[ 0.37235808, -1.9119602 ,  1.6370233 ],
         [ 1.5405345 ,  1.2227023 , -1.7154248 ],
         [ 0.6164763 ,  0.27805877,  0.56488496]]]], dtype=float32)>